In [ ]:
from lib.functions0 import *
import numpy as np
import datetime
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

backend_sim_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_real_jakarta = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

In [ ]:
n_steps=99
time=np.pi
reps = 1
shots = 8000
backend = backend_sim_jakarta
precision=50

In [ ]:
numeric_evolution_matrix = eye(8)

for _ in range(n_steps): # here is computed the evolution operator numerically, with n_steps trotter steps.
    numeric_evolution_matrix=(numeric_evolution_matrix*trotter_step_matrix(time/n_steps)).evalf(precision)

# here are computed the parameters of the gates as described in "decomposition.ipynb" file.
phase_1_1, phase_2_1, phase_1_2, phase_2_2, a1, a2 = get_gates_parameters(initial_state={"110": 1.0}, U=numeric_evolution_matrix)

M1_qc = fixed_magnetization_two_qubit_gate(phase_1_1,phase_1_2,a1)
M2_qc = fixed_magnetization_two_qubit_gate(phase_2_1,phase_2_2,a2)

In [ ]:
numeric_evolution_matrix.evalf(6)

In [ ]:
qr3 = QuantumRegister(3 ,name="q")
qc3 = QuantumCircuit(qr3, name="U")

qc3.append(M1_qc, [qr3[0],qr3[1]])
qc3.append(M2_qc, [qr3[1],qr3[2]])

qc3=transpile(qc3, basis_gates=["cx","rz","x","sx"])
qc3.draw(output="mpl")

In [ ]:
backend_ = Aer.get_backend('unitary_simulator')
job = execute(qc3, backend_, shots=32000)
result = job.result()
A=result.get_unitary(qc3, decimals=10)
Matrix(A)

In [ ]:
qr = QuantumRegister(7 ,name="q")
qc = QuantumCircuit(qr, name="U")


l=0
for k in [5,3,1]:
    if initial_state[l]=='1':
        qc.x(qr[k])
    l+=1

qc.append(qc3, [qr[1],qr[3],qr[5]])

qc.draw(output="mpl")

In [ ]:
qc_check=symmetry_check(type="4copy_check")
qc_check.draw(output="mpl")

In [ ]:
qc=add_symmetry_check(qc, [qr[1],qr[3],qr[5]], [qr[0],qr[2],qr[4],qr[6]], type="4copy_check")
qc.draw(output="mpl")

In [ ]:
qcs_tomo = state_tomography_circuits(qc, [qr[1],qr[3],qr[5]])
qcs_na = state_tomography_circuits(qc, [qr[1],qr[3],qr[5]]) #cirquits without ancillas measurament for the fidelity computation

for qc_iter in qcs_tomo:
    qc_iter.barrier()
    cr_anc=ClassicalRegister(4)
    qc_iter.add_register(cr_anc)
    qc_iter.measure([qr[0],qr[2],qr[4],qr[6]], cr_anc)

In [ ]:
qcs_tomo[0].draw(output="mpl")

In [ ]:
qcs_calibs, meas_calibs = calibration_cirquits("itself", q_anc=[0,2,4,6], N=n_steps, time=time, check="yes", check_type="4copy_check")
state_lables = bin_list(7)

In [ ]:
qcs_calibs[0].draw(output="mpl")

In [ ]:
jobs_evo=execute(qcs_tomo, backend=backend, shots=shots)
job_cal_our=execute(qcs_calibs, backend=backend, shots=shots)
job_cal=execute(meas_calibs, backend=backend, shots=shots)

In [ ]:
meas_fitter_our = CompleteMeasFitter(job_cal_our.result(), state_labels=state_lables)
meas_fitter = CompleteMeasFitter(job_cal.result(), state_labels=state_lables)

U_tilde=meas_fitter_our.cal_matrix

In [ ]:
Matrix(U_tilde)

# da qui in poi non ti fidare (quasi) di niente

In [ ]:
qr_basi = QuantumRegister(7)
qc_basi = QuantumCircuit(qr_basi)

#qc_basi.swap(5,4)
#qc_basi.swap(5,2)
#qc_basi.swap(1,3)
#qc_basi.swap(0,1)

qcs_basi = state_tomography_circuits(qc_basi,[qr_basi[0],qr_basi[1],qr_basi[2]])

qr_aus = QuantumRegister(7 ,name="q")
qc_aus = QuantumCircuit(qr_aus, name="U")

qc_aus.append(qc3, [qr_aus[0],qr_aus[1],qr_aus[2]])
qc_aus=add_symmetry_check(qc_aus, [qr_aus[0],qr_aus[1],qr_aus[2]], [qr_aus[3],qr_aus[4],qr_aus[5],qr_aus[6]], type="4copy_check")

U = matrix_from_cirquit(qc_aus, type="numpy")

C_matrices = []
C= np.matmul(U_tilde, np.asarray(np.matrix(U).getH()))

for qca in qcs_basi:
    
    qca.remove_final_measurements()

    #qca.swap(0,1)
    #qca.swap(1,3)
    #qca.swap(5,2)
    #qca.swap(5,4)

    qca_matrix = np.matrix(matrix_from_cirquit(qca, type="numpy"))
    qca_matrixH = qca_matrix.getH()

    C_aus = np.matmul(qca_matrix, np.matmul(C, qca_matrixH))

    C_matrices.append(np.asarray(C_aus))

In [ ]:
from copy import deepcopy
meas_fitter_ours = []

for C_new in C_matrices:
    meas_fitter_our_aus = deepcopy(meas_fitter_our)
    meas_fitter_our_aus._tens_fitt.cal_matrices[0]=C_new

    meas_fitter_ours.append(meas_fitter_our_aus)



In [ ]:
reps=1

target_state = (One^One^Zero).to_matrix()

fids=np.zeros([reps,4])
fids_mean=np.zeros(4)
fids_dev=np.zeros(4)

for j in range(reps):
    
    #res = jobs_evo_result[j]
    res=jobs_evo.result()

    print(j)
    
    new_res, new_res_nm = mitigate2(res, ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter)
    new_res_our, new_res_nm = mitigate2(res, ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_ours)
    new_res_not_mitigated = mitigate2(res, ancillas_conditions=bin_list(4))

    fids[j,0] = fidelity_count(new_res_not_mitigated, qcs_na, target_state)
    fids[j,1] = fidelity_count(new_res_nm, qcs_na, target_state)
    fids[j,2] = fidelity_count(new_res, qcs_na, target_state)
    fids[j,3] = fidelity_count(new_res_our, qcs_na, target_state)

for i in range(4):
    fids_mean[i]=np.mean(fids[:,i])
    fids_dev[i]=np.std(fids[:,i])

In [ ]:
fids_mean